# AWS API Test

In [5]:
import os, cv2, requests, base64
from flask import Flask, jsonify, request, send_from_directory
from flask_cors import CORS
import numpy as np
from WhiteflyCounting.WFC import whitefly_counting

app = Flask(__name__)
CORS(app)

@app.route('/url_dwnld/<url>')
def url_dwnld(url):
    url = url + "==="
    url = base64.b64decode(url).decode()
    r = requests.get(url, allow_redirects=True)
    open("./whitefly_test.jpg", 'wb').write(r.content)
    Elliptical_Fit_result, ellipse_area = whitefly_counting("./whitefly_test.jpg")
    return str(len(ellipse_area))


@app.route('/upload/', methods=['POST'])
def up_load_big_file():
    img = request.files.get('bigfile')
    img.save("./whitefly_test.jpg")
    Elliptical_Fit_result, ellipse_area = whitefly_counting("./whitefly_test.jpg")
    return str(len(ellipse_area))

@app.route("/download_file")
def download_file():
    # 假设在当前目录
    directory = os.getcwd()  
    return send_from_directory(directory, "Keamns_Elliptical_Fit_Output.jpg", as_attachment=True)


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Aug/2021 22:04:52] "GET /download_file HTTP/1.1" 404 -
127.0.0.1 - - [14/Aug/2021 22:04:53] "POST /upload/ HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2021 22:04:59] "GET /download_file HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2021 22:05:00] "POST /upload/ HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2021 22:05:17] "GET /download_file HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2021 22:05:17] "POST /upload/ HTTP/1.1" 200 -


In [3]:
import base64
url = 'https://img.vim-cn.com/d5/6dbdcf647e5b0a85e2c58e2a20b69d1051625c.png'
base64.b64encode(url.encode())

b'aHR0cHM6Ly9pbWcudmltLWNuLmNvbS9kNS82ZGJkY2Y2NDdlNWIwYTg1ZTJjNThlMmEyMGI2OWQxMDUxNjI1Yy5wbmc='

In [4]:
base64.b64decode('aHR0cHM6Ly9pbWcudmltLWNuLmNvbS9kNS82ZGJkY2Y2NDdlNWIwYTg1ZTJjNThlMmEyMGI2OWQxMDUxNjI1Yy5wbmc========').decode()

'https://img.vim-cn.com/d5/6dbdcf647e5b0a85e2c58e2a20b69d1051625c.png'

In [ ]:
## all code
import os
from flask import Flask, jsonify, render_template, request, url_for, send_from_directory
from werkzeug.utils import secure_filename
import math
import cv2
import numpy as np
from PIL import Image, ImageOps
from flask_cors import CORS
import requests
import base64


## sticky card crop function
def resizeImg(image, height=900):
    h, w = image.shape[:2]
    pro = height / h
    size = (int(w * pro), int(height))
    img = cv2.resize(image, size)
    return img

# canny edge detection
def getCanny(image):
    # 高斯模糊
    binary = cv2.GaussianBlur(image, (11, 11), 2, 2)
    # 边缘检测
    binary = cv2.Canny(binary, 60, 240, apertureSize=3)
    # 膨胀操作 尽量使边缘闭合
    kernel = np.ones((3, 3), np.uint8)
    binary = cv2.dilate(binary, kernel, iterations=1)
    return binary

# findMaxContour
def findMaxContour(image):
    # 寻找边缘
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    # 计算面积
    max_area = 0.0
    max_contour = []
    for contour in contours:
        currentArea = cv2.contourArea(contour)
        if currentArea > max_area:
            max_area = currentArea
            max_contour = contour
    return max_contour, max_area

# get hulk point of parallel
def getBoxPoint(contour):
    # 多边形拟合凸包
    hull = cv2.convexHull(contour)
    epsilon = 0.02 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(hull, epsilon, True)
    approx = approx.reshape((len(approx), 2))
    return approx

# get point value in oringal img
def adaPoint(box, pro):
    box_pro = box
    if pro != 1.0 :
        box_pro = box/pro
    box_pro = np.trunc(box_pro)
    return box_pro

# ordering point
def orderPoints(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)# sum of point value(value_x + value_y)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)# diff between value_y and value_x
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

# calculate point distance
def pointDistance(a, b):
    return int(np.sqrt(np.sum(np.square(a - b))))

# PerspectiveTransform
def warpImage(image, box):
    w, h = pointDistance(box[0], box[1]), \
           pointDistance(box[1], box[2])
    dst_rect = np.array([[0, 0],
                         [w - 1, 0],
                         [w - 1, h - 1],
                         [0, h - 1]], dtype='float32')
    M = cv2.getPerspectiveTransform(box, dst_rect)
    warped = cv2.warpPerspective(image, M, (w, h))
    return warped












In [ ]:
def sticky_card_crop(path):
    image = cv2.imread(path)
    ratio = 900 / image.shape[0]
    img = resizeImg(image)
    binary_img = getCanny(img)
    max_contour, _ = findMaxContour(binary_img)
    boxes = getBoxPoint(max_contour)
    boxes = adaPoint(boxes, ratio)
    boxes = orderPoints(boxes)
    warped = warpImage(image, boxes)
    return warped

In [ ]:
## pading pic
def padding_pic(img):
    img_row_num = 3 - img.shape[0]%3
    img=cv2.copyMakeBorder(img, 0, img_row_num, 0, 0, cv2.BORDER_CONSTANT, value=0)
    return img

In [ ]:
## K-Means, Binarization and Elliptical Fit Code
## K-means Code
def pest_count_by_kmeans_and_binarz(Img, num_clusters=2):
    #Img = cv2.imread(path, 1)
    Img = Img[:,:,0]
    # cv2.GaussianBlur
    blur = cv2.GaussianBlur(Img, (9, 9), 0)
    
    # binarization
    _, otsu = cv2.threshold(blur, 60, 255, cv2.THRESH_BINARY)
    
    # k-means
    data = otsu.reshape((-1,3))
    data = np.float32(data)
    # stop criteria: criteria flag(means one of them), iteration, epsilon
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    # cv2.kmeans input six parameter: source data, cluster number, preset labels, 
    # stop criteria, repeat times, initial center
    # cv2.kmeans return three value: retrurn value type, label of pixel, cluster center
    _,label,_=cv2.kmeans(data, num_clusters, None, criteria, 
                           num_clusters, cv2.KMEANS_RANDOM_CENTERS)
    color = np.uint8([[255, 0, 0],[128, 128, 128]])
    res = color[label.flatten()]
    result = res.reshape((Img.shape))
    return result

In [ ]:
## Elliptical Fit Code
def Elliptical_Fit(kmeans_result, original_file):
    blur = cv2.GaussianBlur(kmeans_result, (9, 9), 0)
    _, otsu = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # cv2.Canny input three value: source img, min value, max value
    binary = cv2.Canny(otsu, 80, 80 * 2)
    # cv2.findContours input three values: source img, detect model(external outline only), output value store type 
    contours,_ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #src = original_file
    ellipse_area = list()
    for c in range(len(contours)):
        if contours[c].size/2 >4:
            # 椭圆拟合
            (cx, cy), (a, b), angle = cv2.fitEllipse(contours[c])
            # 绘制椭圆
            if (cx > 0) & (cy > 0):
                cv2.ellipse(original_file, (np.int32(cx), np.int32(cy)),
                           (np.int32(a/2), np.int32(b/2)), angle, 0, 360, (0, 0, 255), 1, 8)
                ellipse_area.append(round(math.pi*a*b/4,3))
    
    cv2.imwrite("Keamns_Elliptical_Fit_Output.jpg",original_file)
    #src = cv2.cvtColor(original_file, cv2.COLOR_BGR2RGB)
    #plt.imshow(src),plt.xticks([]),plt.yticks([]),plt.show()
    return (original_file, ellipse_area)

In [ ]:
## merge all code
def whitefly_counting(path):
    croped_img = sticky_card_crop(path)
    img = padding_pic(croped_img)
    kmeans_result = pest_count_by_kmeans_and_binarz(img)
    Elliptical_Fit_result, ellipse_area = Elliptical_Fit(kmeans_result, croped_img)
    return Elliptical_Fit_result, ellipse_area

